<a href="https://colab.research.google.com/github/haritmm97/ML-Projects-Portfolio/blob/master/Capstone_polyreg_python_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Import required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from IPython.display import Image
from graphviz import Source
from sklearn import tree
import pydotplus

dfdata = pd.read_csv("/content/drive/My Drive/Machine Learning/Great Learning/Capstone/innercity.csv")

#converting the dayhours to get the year and month on which it was sold
dfdata['dayhours']=dfdata['dayhours'].astype('datetime64[ns]')
dfdata['soldyr']=pd.DatetimeIndex(dfdata['dayhours']).year
dfdata['soldmonth']=pd.DatetimeIndex(dfdata['dayhours']).month

#dropping dayhours column
house_data=dfdata.drop('dayhours',axis=1)

#Removing yr_renovated column
house_dat=house_data.drop('yr_renovated',axis=1)
#Adding a column to flag the renovation
house_dat['renovated']=0
house_dat['renovated'].loc[((house_dat['living_measure']<house_dat['living_measure15'])|(house_dat['living_measure']>house_dat['living_measure15'])|(house_dat['lot_measure']<house_dat['lot_measure15'])|(house_dat['lot_measure']>house_dat['lot_measure15']))]=1

#Removing living_measure 15 and lot_measure15 columns
house_dat1=house_dat.drop(['living_measure15','lot_measure15'],axis=1)

#creating new variable "cost_persqft" to main df
house_dat1['cost_persqft'] = house_dat1['price']/house_dat1['total_area']

house_dat1['median_cost_persqft'] = house_dat1.groupby('zipcode')['cost_persqft'].transform(np.median)

costpsft_zip = house_dat1[['zipcode', 'median_cost_persqft']]

costpsft_zip = costpsft_zip.drop_duplicates(subset='zipcode', keep="last")

house_dat1=house_dat1.drop(['cost_persqft', 'median_cost_persqft'],axis=1)

house_dat1 = pd.merge(house_dat1, costpsft_zip, on='zipcode')

#removing the cid column as it doesnt conribute to price
house_dat2=house_dat1.drop(['cid'],axis=1)

house_dat2['ceil'] = pd.Categorical(house_dat2.ceil)
house_dat2['coast'] = pd.Categorical(house_dat2.coast)
house_dat2['sight'] = pd.Categorical(house_dat2.sight)
house_dat2['condition'] = pd.Categorical(house_dat2.condition)
house_dat2['quality'] = pd.Categorical(house_dat2.quality)
house_dat2['furnished'] = pd.Categorical(house_dat2.furnished)
house_dat2['renovated'] = pd.Categorical(house_dat2.renovated)
house_dat2['zipcode'] = pd.Categorical(house_dat2.zipcode)

house_dat2.drop(["soldyr", "soldmonth", "lat", "long"], axis = 1, inplace = True)

house_dat2['current_yr'] = 2015
# creating "house_age" feature by subracting yr_built to 2015
house_dat2['house_old'] = house_dat2['current_yr'] - house_dat2['yr_built']
house_dat2.drop(["current_yr", "yr_built"], axis = 1, inplace = True)
house_dat2.head()

df_zip = pd.read_csv("/content/drive/My Drive/Machine Learning/Great Learning/Capstone/zipcode3.csv")
df_zip.drop(["region", "link1", "link2"], axis = 1, inplace = True)

df_zip["Total_population"] = df_zip["Total_population"].str.replace(",","").astype(float)
df_zip["Housing_units"] = df_zip["Housing_units"].str.replace(",","").astype(float)
df_zip["Density"] = df_zip["Density"].str.replace(",","").astype(float)
df_zip["White_Population"] = df_zip["White_Population"].str.replace(",","").astype(float)
df_zip["Median_Income"] = df_zip["Median_Income"].str.replace(",","").astype(float)

df_zip['white_perc'] = df_zip['White_Population']/df_zip['Total_population']
df_zip['per_capita_house'] = df_zip['Housing_units']/df_zip['Total_population']

house_dat2 = pd.merge(house_dat2, df_zip, on='zipcode')

house_dat2.drop(["Total_population", "Housing_units", "White_Population"], axis = 1, inplace = True)

house_dat2['zipcode'] = pd.Categorical(house_dat2.zipcode)

# Fetch all numeric features
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in house_dat2.columns:
    if house_dat2[i].dtype in numeric_dtypes:
        numeric.append(i)

from scipy.stats import zscore
house_dat2[numeric] = house_dat2[numeric].apply(zscore)

from sklearn.model_selection import train_test_split
X=house_dat2.drop('price',axis=1) #Indepedent feature columns
y=house_dat2['price'] # dependent feature column
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
polymodel = PolynomialFeatures(degree=2,interaction_only=True)
poly_x_train = polymodel.fit_transform(x_train)
poly_x_test = polymodel.fit_transform(x_test)
linear_polymodel = linear_model.LinearRegression()
linear_polymodel.fit(poly_x_train,y_train)
#predict using test
poly_y_predict=linear_polymodel.predict(poly_x_test)
#predict using training data
poly_y_predict_trn=linear_polymodel.predict(poly_x_train)

#polynomial regression model score
model_score_polytrn=linear_polymodel.score(poly_x_train,y_train)
print("Polynomial regresion model score using training data:",model_score_polytrn)
model_score_poly=linear_polymodel.score(poly_x_test,y_test)
print("Polynomial regresion model score using test data:",model_score_poly)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Polynomial regresion model score using training data: 0.8809290330108911
Polynomial regresion model score using test data: 0.8718794021889589
